# Pydantic Basic
- work with data in OOP manner 
- fields and methods
- validate data 
- serialize -OOP instance -> jason
- deserialize - Jason -> OOP object 

## Start without pydantic 
how to naively create validation for a Person class
- error handling 
- property 
    - getter
    - setter (put in validation here)

In [1]:
class Person:
    def __init__(self, name, gender, age):
        self.name = name 
        self.geneder = gender 
        self.age = age 
p1 = Person("Gauss", "M", 73)
# defaukt __rep__ for the Person class
p1
        

In [2]:
class Person:
    def __init__(self, name, gender, age):
        self.name = name 
        self.gender = gender 
        self.age = age 

    def __repr__(self):
        return f"Person(name={self.name}, gender={self.gender}, age={self.age})"

p1 = Person("Gauss", "M", 73)
p1



Person(name=Gauss, gender=M, age=73)

In [3]:
p2 = Person("Dadda", "F", 3)
p2
 

Person(name=Dadda, gender=F, age=3)

In [4]:
p1.age, p1.name

(73, 'Gauss')

In [5]:
p2.age, p2.name

(3, 'Dadda')

create a totally invalid Person

In [6]:
Person(3.1455342, 2.715, "twentyFive")

Person(name=3.1455342, gender=2.715, age=twentyFive)

It take it as everiting is totally normal.
So this should give an error actually 


Some other programmering lenguages have static typing or hard typing
on python we have a dinamically typed language

in herd type language 
``` 
int age;
string name;
string gender;
```
it will give error when langauges have static typing or hard typing 
in 

In [7]:
class Person:
    def __init__(self, name: str, gender:str, age: int):
        self.name = name 
        self.gender = gender 
        self.age = age 

    def __repr__(self):
        return f"Person(name={self.name}, gender={self.gender}, age={self.age})"
# in python types are not enforced 
Person(3.14, "M", 33)

Person(name=3.14, gender=M, age=33)

Add Validation

In [8]:
class Person:
    def __init__(self, name: str, gender:str, age: int):
        self.name = name 
        self.gender = gender 
        self.age = age 

    @property
    def age(self):
        print("getter called")
        return self._age
    
    @age.setter
    def age(self, age: int):
        print ("setter called, validation age code here ")
        if not isinstance(age, int):
            raise TypeError (f"age must be an int, not type {type(age)}")
        if not 0<=age<=125:
            raise ValueError (f"age must be between 0 and 125, not {age}")
        self._age = age


    def __repr__(self):
        return f"Person(name={self.name}, gender={self.gender}, age={self.age})"
# in python types are not enforced 
#p3 = Person("Henry", gender = "M", age = 32)
#p3
try:
    p4 = Person("henry", gender= "M", age = "fem")
except TypeError as err:
    print(err)
try:
    p5 = Person("henry", gender= "M", age = -5)
except ValueError as err:
    print(err)



setter called, validation age code here 
age must be an int, not type <class 'str'>
setter called, validation age code here 
age must be between 0 and 125, not -5


## Pydantic for same example

In [9]:
from pydantic import BaseModel, ValidationError
class Person(BaseModel):
    name: str
    gender: str 
    age: int 
# more fast and less writing code but -2 is not a problem here 
p5 = Person(name="Babba", gender="F", age=-2)
p5


Person(name='Babba', gender='F', age=-2)

In [10]:
try:
    Person(name=1.41, gender=28, age="five")
except ValidationError as err:
    print(err)

3 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=1.41, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=28, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='five', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [16]:
from pydantic import BaseModel, ValidationError
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal ["M","F"] = Field(description= "gender, valid values are 'M' and 'F'")
    age: int = Field(0, gt=-1, lt=126, description="age of a human person")

try:
    p6 = Person(name="Bidi", gender="M", age=100)
    print(p6)
except ValidationError as err:
    print(err)



name='Bidi' gender='M' age=100


## Serialize pydantic model

make it into json


In [17]:
p6 = Person(name ="Doddo", gender= "F", age=5)
p6

Person(name='Doddo', gender='F', age=5)

In [18]:
type(p6.model_dump())

dict

In [19]:
p6.model_dump()

{'name': 'Doddo', 'gender': 'F', 'age': 5}

In [21]:
p6.model_dump_json()

'{"name":"Doddo","gender":"F","age":5}'

In [22]:
print(p6.model_dump_json(indent=3))

{
   "name": "Doddo",
   "gender": "F",
   "age": 5
}


In [23]:
with open("person.json", "w") as file:
    file.write(p6.model_dump_json())

## deserialization


In [25]:
with open("person.json", "r") as file:
    person_data = file.read()
p7 = Person.model_validate_json(person_data)
p7

Person(name='Doddo', gender='F', age=5)